In [1]:
from PIL import Image
import numpy as np
import mcpi.minecraft as minecraft
import mcpi.entity as entity
import mcpi.block as block
import random
import math
from mcpi.vec3 import Vec3
import time

mc = minecraft.Minecraft.create(address='192.168.1.38')

ConnectionRefusedError: [Errno 111] Connection refused

In [155]:
class Shaihulud:
    def __init__(self, mc, x, y, z, len=5, speed=1):
        self.mc = mc
        self.speed = speed
        self.body = []
        self.steps_in_dir = len+1
        self.current_steps = 0

        # Speed in blocks per s
        for i in np.arange(0, len, 1):
            self.body.append((x, y, z+i))
            mc.setBlock(x, y, z+i, block.WOOL_ORANGE)

        self.choose_dir()

    def update(self, dt):
        steps = round(dt/self.speed)
        for s in np.arange(0, steps, 1):
            self.step()

    def choose_dir(self):
        progressions = set([
            (0,0,1),
            (0,0,-1),
            (0,1,0),
            (0,-1,0),
            (1,0,0),
            (-1,0,0),
        ])
        heading = (
            round(self.body[1][0]-self.body[0][0]),
            round(self.body[1][1]-self.body[0][1]),
            round(self.body[1][2]-self.body[0][2])
        )
        progressions = progressions.difference(set([heading]))

        head_wh = mc.getHeight(self.body[0][0], self.body[0][2])
        if self.body[0][1]>=head_wh:
            progressions = progressions.difference(set([(0,1,0)]))
        #if self.body[0][1]<head_wh-10:
        #    progressions = progressions.difference(set([(0,-1,0)]))

        self.current_steps = 0
        self.dir = random.choice(list(progressions))

    def step(self):

        tail = self.body.pop()
        mc.setBlock(*tail, block.AIR)

        new_head = (
            self.body[0][0]+self.dir[0],
            self.body[0][1]+self.dir[1],
            self.body[0][2]+self.dir[2],
        )
        self.body.insert(0, new_head)
        mc.setBlock(*new_head, block.WOOL_ORANGE)

        self.current_steps += 1
        if self.current_steps>=self.steps_in_dir:
            self.choose_dir()

In [156]:
pp = mc.player.getPos()

shs = []
for i in np.arange(0, 1, 1):
    shs.append(Shaihulud(mc, pp.x+5, pp.y, pp.z, len=10))

In [157]:
while True:
    time.sleep(0.1)
    for sh in shs:
        sh.step()

ValueError: Socket got closed